*   **Año:** [2024]
*   **Alumno/a:** [BOLIVAR, Agustin]
*   **Legajo:** [45822853]

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [ ]:
import pydantic as pyd

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [ ]:
from pydantic import BaseModel, StrictInt, ValidationError, validator
from datetime import date

class Client(BaseModel):
    dni: StrictInt
    nacionalidad: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre
todos sus campos.

In [ ]:
cliente = Client(dni=45_333_111, nacionalidad="Argentina")
cliente

Client(dni=45333111, nacionalidad='Argentina')

Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [ ]:
cliente2 = Client(dni="45_333_111", nacionalidad="Argentina")
cliente2

ValidationError: 1 validation error for Client
dni
  Input should be a valid integer [type=int_type, input_value='45_333_111', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_type

Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [ ]:
class Client2(BaseModel):
    dni: StrictInt
    nacionalidad: str
    registro: date

    @validator("dni")
    def validar_dni(cls,value):
        if value < 0:
          raise ValueError(f"dni '{value}' no es válido. Debe ser mayor a 0.")
        return value

    @validator("nacionalidad")
    def validar_nacionalidad(cls, value):
        # Lista de nacionalidades válidas
        nacionalidades_validas = ["Argentina", "Brasil", "Chile", "Uruguay", "Paraguay"]
        if value not in nacionalidades_validas:
            raise ValueError(f"Nacionalidad '{value}' no es válida. Debe estar en {nacionalidades_validas}.")
        return value

    @validator("registro")
    def validar_fecha_registro(cls, value):
        if value > date.today():
            raise ValueError("La fecha de registro no puede ser en el futuro.")
        return value

<ipython-input-19-facf60d059df>:6: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator("dni")
<ipython-input-19-facf60d059df>:12: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator("nacionalidad")
<ipython-input-19-facf60d059df>:20: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more d

In [ ]:
cliente3 = Client2(dni=39755010, nacionalidad="Argentina", registro=date(2024, 11, 19))

Client2(dni=39755010, nacionalidad='Argentina', registro=datetime.date(2024, 11, 19))

In [ ]:
cliente4 = Client2(dni=39755010, nacionalidad="Argentina", registro=date(2024, 11, 19))

In [ ]:
cliente5 = Client2(dni=39755010, nacionalidad="Argentina", registro=date(2024, 11, 19))

In [ ]:
cliente6 = Client2(dni=-39755010, nacionalidad="Argentina", registro=date(2024, 1, 1)) # error de dni negativo
#cliente7 = Client2(dni=39755010, nacionalidad="Turquia", registro=date(2024, 1, 1)) #error de nacionalidad
#cliente8 = Client2(dni=39755010, nacionalidad="Argentina", registro=date(2025, 1, 1)) #error de fecha

ValidationError: 1 validation error for Client2
dni
  Value error, dni '-39755010' no es válido. Debe ser mayor a 0. [type=value_error, input_value=-39755010, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [ ]:
!pip install pandera

In [ ]:
from datetime import date
import pandera as pa
import pandas as pd

class Client3:
    def __init__(self, dni: int, nacionalidad: str, fecha_registro: date):
        self.dni = dni
        self.nacionalidad = nacionalidad
        self.fecha_registro = fecha_registro

        self.validate_client()

    def __repr__(self):
        return (f"Client3(DNI={self.dni}, Nacionalidad='{self.nacionalidad}', "
                f"Fecha de Registro={self.fecha_registro})")

    # Esquema de validación con pandera
    client_schema = pa.DataFrameSchema({
        "dni": pa.Column(int, checks=pa.Check.ge(0)),  # DNI positivo
        "nacionalidad": pa.Column(
            str, checks=pa.Check.isin(["Argentina", "Chile", "Brasil", "Uruguay", "Paraguay"])
        ),
        "fecha_registro": pa.Column(
            pd.Timestamp, checks=pa.Check.le(pd.Timestamp(date.today()))  # Fecha no futura
        )
    })

    def validate_client(self):
        df_client = pd.DataFrame([{
            "dni": self.dni,
            "nacionalidad": self.nacionalidad,
            "fecha_registro": pd.Timestamp(self.fecha_registro)
        }])

        self.client_schema.validate(df_client)


In [ ]:
#INVALIDO
try:
  cliente9 = Client3(39755010, "Peru", date(2023, 11, 14))
  print(cliente3)
except:
  print("error")

error


In [ ]:
#VALIDO
cliente10 = Client3(39755020, "Argentina", date(2023, 11, 14))
print(cliente10)


Client3(DNI=39755020, Nacionalidad='Argentina', Fecha de Registro=2023-11-14)
